In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
%matplotlib inline

from datetime import timedelta

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
                                                            
import xgboost as xgb
from sklearn import preprocessing
from sklearn.feature_selection import RFE, SelectKBest, chi2

In [2]:
df_phone = pd.read_csv("phone_dataset .csv", error_bad_lines=False)

b'Skipping line 821: expected 40 fields, saw 41\nSkipping line 6060: expected 40 fields, saw 41\nSkipping line 6663: expected 40 fields, saw 41\n'


In [3]:
df_phone = df_phone[['brand','model','approx_price_EUR']].drop_duplicates().dropna()

In [4]:
df_phone['model'] = df_phone['model'].str.lower()
df_phone['brand'] = df_phone['brand'].str.lower()
df_phone.drop_duplicates('model', inplace = True)

In [5]:
df_phone.head()

,brand,model,approx_price_EUR
0,acer,iconia talk s,170.0
1,acer,liquid z6 plus,250.0
2,acer,liquid z6,120.0
3,acer,iconia tab 10 a3-a40,230.0
4,acer,liquid x2,230.0


In [6]:
df_phone['model'] = df_phone['brand'] + ' ' + df_phone['model']

In [7]:
df_phone = df_phone[['model','approx_price_EUR']]

In [8]:
df_phone.head()

,model,approx_price_EUR
0,acer iconia talk s,170.0
1,acer liquid z6 plus,250.0
2,acer liquid z6,120.0
3,acer iconia tab 10 a3-a40,230.0
4,acer liquid x2,230.0


In [9]:
df_events = pd.read_csv('../../data/events_up_to_01062018.csv')

/home/santi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
df_todas_las_personas = df_events[['person']].drop_duplicates()

In [24]:
df_events2 = df_events[df_events['event'] == 'viewed product']
df_events2 = df_events2[['model']]

In [25]:
df_events2['model'] = df_events2['model'].str.lower()

In [26]:
df_events2 = df_events2.drop_duplicates('model')

In [27]:
df_events2.shape

(199, 1)

In [28]:
def agregarApple(s):
    if('iphone' in s):
        return 'apple '+s
    return s

def borrarApple(s):
    if('apple 'in s):
        return s[6:]
    return s

In [29]:
df_events2['model'] = df_events2['model'].apply(agregarApple)

In [30]:
df_phone['model'].value_counts()

htc desire 700 dual sim         1
sony smartwatch 2 sw2           1
samsung a886 forever            1
nokia 6303 classic              1
lenovo s580                     1
nokia n80                       1
samsung p6800 galaxy tab 7.7    1
leeco le 2                      1
samsung galaxy s5 (usa)         1
zte nubia z17 mini              1
samsung galaxy grand prime      1
micromax x288                   1
lg stylus 2 plus                1
samsung u100                    1
motorola motoluxe xt389         1
micromax x450                   1
htc advantage x7500             1
nokia xl                        1
lg cookie lite t300             1
allview e4                      1
spice mi-720                    1
sony smartwatch 3 swr50         1
huawei ideos s7 slim cdma       1
samsung i400                    1
samsung i8530 galaxy beam       1
blackberry classic              1
lenovo tab3 10                  1
leeco le pro3                   1
htc rhyme                       1
xolo q2500    

In [31]:
df_events2['model'].value_counts()

samsung galaxy tab e 7 wi-fi                   1
samsung galaxy s6 flat                         1
samsung galaxy a9 pro 2016                     1
samsung galaxy j7 2016 metal                   1
samsung gear s2                                1
sony xperia z2                                 1
samsung galaxy j5 2016 metal                   1
samsung galaxy s6 edge plus                    1
samsung galaxy tab pro 10.1 wi-fi              1
apple iphone 6 plus                            1
apple iphone 6                                 1
apple iphone 7 plus                            1
ipad mini 3 wi-fi                              1
samsung galaxy note 3 neo duos                 1
samsung galaxy j5                              1
ipad mini wi-fi                                1
samsung galaxy j2 4g duos tv                   1
lg  x screen                                   1
apple iphone 5                                 1
samsung galaxy mega duos                       1
samsung galaxy tab 3

In [32]:
celulares_con_precio = pd.merge(df_events2,df_phone, on = 'model', how = 'left')

In [33]:
celulares_con_precio = celulares_con_precio.fillna(0)

In [34]:
celulares_con_precio['model'] = celulares_con_precio['model'].apply(borrarApple)

In [35]:
celulares_con_precio

,model,approx_price_EUR
0,samsung galaxy j7 prime,280.0
1,iphone 5s,330.0
2,iphone 6s,620.0
3,iphone 7,760.0
4,lg g4 h818p,0.0
5,iphone 6,540.0
6,iphone 6 plus,700.0
7,motorola moto g5 plus,280.0
8,motorola moto g4 plus,230.0
9,ipad air wi-fi,0.0


In [42]:
df_events['model'] = df_events['model'].str.lower()
df_events3 = pd.merge(df_events,celulares_con_precio,on = 'model',how = 'inner')

In [37]:
df_events3.drop_duplicates('person').shape

(38255, 24)

In [43]:
df_events3 = df_events3[df_events3['event'] == 'viewed product']

In [48]:
df_events3 = df_events3[['person','approx_price_EUR']].groupby(['person']).sum()

In [52]:
df_events3.reset_index(inplace = True)

In [54]:
df_events3 = pd.merge(df_todas_las_personas,df_events3,how = 'left', on = 'person').fillna(0)

In [61]:
df_events3.head()

,person,approx_price_EUR
0,4886f805,1120.0
1,ad93850f,6830.0
2,0297fc1e,236130.0
3,2d681dd8,5710.0
4,cccea85e,105710.0


In [64]:
df_events_util = df_events.drop_duplicates(['person','model'])

In [65]:
df_cant_visito = (df_events_util[df_events_util['event'] == 'viewed product'])[['person','model']].groupby('person').count()

In [69]:
df_cant_visito.reset_index(inplace = True)

In [70]:
df_cant_visito = pd.merge(df_todas_las_personas,df_cant_visito, how = 'left', on = 'person').fillna(0)
df_cant_visito.head()

,person,model
0,4886f805,1.0
1,ad93850f,3.0
2,0297fc1e,15.0
3,2d681dd8,2.0
4,cccea85e,57.0


In [73]:
df_cant_visito.columns = ['person','cantidadCelularesDistintosVisitados']

In [80]:
df_cant_visito.head()

,person,cantidadCelularesDistintosVisitados
0,4886f805,1.0
1,ad93850f,3.0
2,0297fc1e,15.0
3,2d681dd8,2.0
4,cccea85e,57.0


In [81]:
df_events3.columns = ['person','sumaDePreciosCelularesVisitados']

In [83]:
df_events3.head()

,person,sumaDePreciosCelularesVisitados
0,4886f805,1120.0
1,ad93850f,6830.0
2,0297fc1e,236130.0
3,2d681dd8,5710.0
4,cccea85e,105710.0


In [84]:
df_train = pd.merge(df_events3,df_cant_visito,on = 'person', how = 'inner')

In [86]:
df_train.head()

,person,sumaDePreciosCelularesVisitados,cantidadCelularesDistintosVisitados
0,4886f805,1120.0,1.0
1,ad93850f,6830.0,3.0
2,0297fc1e,236130.0,15.0
3,2d681dd8,5710.0,2.0
4,cccea85e,105710.0,57.0


In [89]:
df_train.to_csv('Santi - FeaturePreciosCelularesYExtra',index=False)